# Stats

This notebook loads some data, reports some simple descriptive statistics (means, standard deviationsm etc) and show a number of useful plots (scatter plots, histograms, time series plots).

Most of the descriptive stats use built-in Julia commands. The plots reply on the Plots package and the pdf and quantiles are from the Distributions package (see https://github.com/JuliaStats/Distributions.jl).

## Load Packages

In [1]:
using Distributions            #distributions, random numbers, etc.
#using Dates, DelimitedFiles   #Julia 0.7
                         
include("printmat.jl")   #just a function for prettier matrix printing

printmatDate

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Load Data from a csv File

The following is a portion of MyData.csv:

```
date,Mkt-RF,RF,SmallGrowth
197901,4.18,0.77,10.96
197902,-3.41,0.73,-2.09
197903,5.75,0.81,11.71
197904,0.05,0.8,3.27
```

In [3]:
x = readdlm("Data/MyData.csv",',',skipstart=1)  #reading the csv file    
                                                #skip 1st line
println("\nfirst four lines of x:") 
printmat(x[1:4,:])


first four lines of x:
197901.000     4.180     0.770    10.960
197902.000    -3.410     0.730    -2.090
197903.000     5.750     0.810    11.710
197904.000     0.050     0.800     3.270



### Creating Variables

In [4]:
ym  = round.(Int,x[:,1])        #yearmonth, like 200712
Rme = x[:,2]                    #picking out the second column
Rf  = x[:,3]                   
R   = x[:,4] .+                 
      0.0                       #commands continue on the next line
Re  = R - Rf                    #do R .- Rf if R has several columns

println("first 4 obs of Rme and Re")
printmat([Rme[1:4,:] Re[1:4,:]])

df = Dates.DateFormat("yyyymm")  #precompile the date format
dN = Date.(string.(ym),df)       #convert to string and then to Julia Date
printmat(dN[1:4])

first 4 obs of Rme and Re
     4.180    10.190
    -3.410    -2.820
     5.750    10.900
     0.050     2.470

1979-01-01
1979-02-01
1979-03-01
1979-04-01



# Some Descriptive Statistics

## Means and Standard Deviations

The next few cells estimate means, standard deviations, covarances and correlations of the variables Rme (US equity market excess return) and Re (excess returns for a segment of the market, small growth firms). 

In [5]:
μ = mean([Rme Re],1)    #,1 to calculate average along a column, gives a row vector
σ = std([Rme Re],1)     #do \sigma[Tab] to get σ

println("            Rme       Re")
printlnPs("means: ",μ)  #for more stat functions, see the package StatsBase.jl
printlnPs("std:   ",σ)

            Rme       Re
means:      0.602     0.303
std:        4.604     8.572


## Covariances and Correlations

In [6]:
println("\n","cov([Rme Re]): ")          
printmat(cov([Rme Re]))

println("\n","cor([Rme Re]): ")          
printmat(cor([Rme Re]))


cov([Rme Re]): 
    21.197    28.426
    28.426    73.475


cor([Rme Re]): 
     1.000     0.720
     0.720     1.000



## OLS

A simple linear regression
$
y = x'b + u
$,
where $x=[1;R^e_m]$.

Clearly, the first element of b is the intercept and the second element is the slope coefficient.

The GLM package (not used here) has powerful regression methods. See https://github.com/JuliaStats/GLM.jl.

In [7]:
c   = ones(size(Rme,1))         #a vector with ones, no. rows from variable
x   = [c Rme]                   #x is a Tx2 matrix
y   = copy(Re)                  #to get standard OLS notation

b2   = inv(x'x)*x'y             #OLS according to a textbook
b  = x\y                        #also OLS, quicker and numerically more stable
u   = y - x*b                   #OLS residuals
R2a = 1 - var(u)/var(y)         #R2, but that name is already taken

Covb  = inv(x'x)*var(u)         #covariance matrix of b estimates
Stdb  = sqrt.(diag(Covb))       #std of b estimates
tstat = (b .- 0)./Stdb          #t-stats, replace 0 with your null hypothesis 

println("OLS coefficients, regressing Re on constant and Rme, different calculations")
printmat([b b2])                
printlnPs("R2: ",R2a) 
printlnPs("no. of observations: ",size(Re,1))

println("\ncoeffs and t-stats")
printmat([b tstat])

OLS coefficients, regressing Re on constant and Rme, different calculations
    -0.504    -0.504
     1.341     1.341

R2:      0.519
no. of observations:        388

coeffs and t-stats
    -0.504    -1.656
     1.341    20.427



# Drawing Random Numbers and Finding Critical Values

## Random Numbers: Independent Variables

In [8]:
T = 100
x = randn(T,2)    #T x 2 matrix, N(0,1) distribution

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -0.072    -0.026
     0.921     0.954

covariance and correlation matrices:
     0.848    -0.065
    -0.065     0.910

     1.000    -0.074
    -0.074     1.000



## Random Numbers: Correlated Variables

In [9]:
μ = [-1;10]
Σ = [1 0.5;
     0.5 2]

T = 100
x = rand(MvNormal(μ,Σ),T)'  #random numbers, T x 2, drawn from bivariate N(μ,Σ)

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -1.190     9.787
     1.007     1.426

covariance and correlation matrices:
     1.015     0.365
     0.365     2.034

     1.000     0.254
     0.254     1.000



## Quantiles ("critical values") of Distributions

In [10]:
N05     = quantile(Normal(0,1),0.05)            #from the Distributions package
Chisq05 = quantile(Chisq(5),0.95)

println("\n","5th percentile of N(0,1) and 95th of Chisquare(5)")      #lots of statistics functions
printmat([N05 Chisq05])


5th percentile of N(0,1) and 95th of Chisquare(5)
    -1.645    11.070



# Statistical Plots

In [11]:
xTicksLoc = Dates.value.([Date(1980);Date(1990);Date(2000);Date(2010)])
xTicksLab = ["1980";"1990";"2000";"2010"]   #crude way of getting the tick marks right

plot3a = plot(dN,Rme,color=:blue,legend=false,xticks=(xTicksLoc,xTicksLab))
plot!(ylim=(-25,25))
title!("Time series plot: monthly US equity market excess return")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 Time series plot: monthly US equity market excess return 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 51.5968,174.87 52.9884,227.137 54.2453,164.058 55.6369,203.31 56.9836,218.667 58.3752,176.936 59.722,198.628 61.1136,164.403 62.5052,208.406 63.8519,259.71 
 65.2435,166.675 66.5902,190.777 67.9818,163.989 69.3734,209.095 70.6752,294.761 72.0668,176.316 73.4135,167.846 74.8051,181.894 76.1518,159.513 77.5434,191.81 
 78.935,188.505 80.2817,196.355 81.6733,137.959 83.02,236.365 84.4116,238.431 85.8032,200.349 87.0602,180.172 88.4518,218.874 89.7985,202.209 91.1901,219.975 
 92.5368,214.329 93.9284,251.239 95.32,256.129 96.6667,170.531 98.0583,179.484 99.405,228.997 100.797,227.206 102.188,245.179 103.445,217.359 104.837,181.618 
 106.183,230.374 107.575,226.724 108.922,225.002 110.313,126.941 111.705,195.598 113.052,126.046 114.443,172.253 115.79,198.283 117.182,179.553 118.573,187.128 
 119.83,184.098 121.222,157.447 122.568,199.316 123.96,182.238 125.307,230.512 126.698,206.478 128.09,197.801 129.437,228.17 130.828,188.092 132.175,215.912 
 133.567,217.841 134.958,235.47 136.26,199.454 137.652,207.511 138.998,245.042 140.39,192.705 141.737,223.487 143.128,131.761 144.52,209.302 145.866,210.61 
 147.258,216.05 148.605,191.741 149.996,149.115 151.388,196.011 152.645,209.095 154.037,210.128 155.383,169.774 156.775,195.667 158.122,208.131 159.513,210.748 
 160.905,235.194 162.251,177.555 163.643,160.202 164.99,178.451 166.381,200.762 167.773,157.378 169.03,170.669 170.421,212.676 171.768,172.046 173.16,197.457 
 174.506,248.347 175.898,161.235 177.29,261.156 178.636,172.873 180.028,195.942 181.375,225.209 182.766,118.057 184.158,173.63 185.415,190.571 186.806,218.392 
 188.153,202.76 189.545,176.867 190.891,176.385 192.283,181.343 193.675,221.077 195.021,363.005 196.413,255.853 197.76,157.929 199.151,174.732 200.543,171.22 
 201.845,218.116 203.236,199.247 204.583,206.891 205.975,171.564 207.321,212.194 208.713,226.999 210.105,182.307 211.451,195.735 212.843,218.874 214.19,193.463 
 215.581,161.923 216.973,219.149 218.23,193.463 219.621,175.076 220.968,182.032 222.36,211.918 223.706,155.381 225.098,193.532 226.489,209.164 227.836,228.514 
 229.228,196.149 230.574,195.253 231.966,255.853 233.358,197.319 234.615,191.466 236.006,227.895 237.353,147.118 238.745,210.885 240.091,214.811 241.483,271.485 
 242.874,244.835 244.221,216.945 245.613,162.337 246.959,187.472 248.351,173.424 249.743,154.762 251,186.783 252.391,205.032 253.738,178.864 255.13,236.847 
 256.476,174.801 257.868,188.367 259.259,214.397 260.606,194.289 261.998,232.027 263.344,132.725 264.736,206.822 266.128,196.355 267.429,222.317 268.821,196.631 
 270.168,201.176 271.559,219.149 272.906,178.313 274.298,219.769 275.689,196.906 277.036,197.664 278.428,177.555 279.774,193.325 281.166,196.562 282.557,201.451 
 283.814,188.092 285.206,222.799 286.553,184.786 287.944,201.658 289.291,205.858 290.683,178.175 292.074,205.032 293.421,192.705 294.813,217.496 296.159,191.81 
 297.551,183.684 298.942,221.766 300.199,237.054 301.591,198.972 302.938,199.385 304.329,225.002 305.676,184.511 307.068,176.867 308.459,218.874 309.806,196.286 
 311.198,231.82 312.544,198.008 313.936,192.499 315.327,179.139 316.584,188.229 317.976,189.469 319.323,183.96 320.714,185.406 322.061,178.657 323.453,200.487 
 324.844,181.55 326.191,214.673 327.582,177.142 328.929,196.562 330.321,187.265 331.712,195.116 333.014,198.834 334.406,189.262 335.753,188.092 337.144,212.125 
 338.491,243.802 339.882,184.098 341.274,170.187 342.621,197.113 344.012,161.304 345.359,214.673 346.751,169.912 348.142,207.098 349.399,237.536 350.791,177.418 
 352.137,157.723 353.529,175.834 354.876,153.935 356.267,231.476 357.659,166.4 359.006,230

In [12]:
plot3b = scatter(Rme,Re,color=:blue,legend=false)
plot!([-40;60],[-40;60],color=:black)
plot!(xlim=(-40,40),ylim=(-40,60))
title!("Scatter plot: two monthly return series (and 45 degree line)")
xlabel!("Market excess return, %")
ylabel!("Excess returns on small growth stocks, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 Scatter plot: two monthly return series (and 45 degree line) 
 
 
 Market excess return, % 
 
 
 Excess returns on small growth stocks, %

In [13]:
xGrid = -25:0.1:15
pdfX   = pdf.(Normal(mean(Rme),std(Rme)),xGrid) #the N(μ,σ) pdf
                                        #"Distributions" wants σ, not σ^2

histogram(Rme,bins = -25:1:15,normalized=true,label="histogram")     #normalized to have area=1
plot!(xGrid,pdfX,linewidth=3,label="fitted N()")
title!("Histogram: monthly US equity market excess return")
xlabel!("Market excess return, %")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 Histogram: monthly US equity market excess return 
 
 
 Market excess return, % 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 60.5404,360.065 61.802,360.065 63.0636,360.065 64.3252,360.065 65.5868,360.065 66.8484,360.065 68.11,360.065 69.3715,360.065 70.6331,360.065 71.8947,360.065 
 73.1563,360.065 74.4179,360.065 75.6795,360.065 76.9411,360.065 78.2027,360.065 79.4643,360.065 80.7258,360.065 81.9874,360.065 83.249,360.065 84.5106,360.065 
 85.7722,360.065 87.0338,360.065 88.2954,360.065 89.557,360.065 90.8186,360.065 92.0801,360.064 93.3417,360.064 94.6033,360.064 95.8649,360.064 97.1265,360.064 
 98.3881,360.064 99.6497,360.064 100.911,360.063 102.173,360.063 103.434,360.063 104.696,360.063 105.958,360.062 107.219,360.062 108.481,360.062 109.742,360.061 
 111.004,360.061 112.266,360.06 113.527,360.06 114.789,360.059 116.05,360.059 117.312,360.058 118.574,360.057 119.835,360.056 121.097,360.055 122.358,360.054 
 123.62,360.053 124.881,360.052 126.143,360.051 127.405,360.049 128.666,360.048 129.928,360.046 131.189,360.044 132.451,360.042 133.713,360.04 134.974,360.037 
 136.236,360.034 137.497,360.031 138.759,360.028 140.021,360.024 141.282,360.02 142.544,360.016 143.805,360.012 145.067,360.006 146.328,360.001 147.59,359.995 
 148.852,359.989 150.113,359.982 151.375,359.974 152.636,359.966 153.898,359.957 155.16,359.947 156.421,359.936 157.683,359.925 158.944,359.913 160.206,359.899 
 161.468,359.885 162.729,359.869 163.991,359.852 165.252,359.834 166.514,359.815 167.775,359.794 169.037,359.771 170.299,359.746 171.56,359.72 172.822,359.692 
 174.083,359.661 175.345,359.628 176.607,359.593 177.868,359.555 179.13,359.514 180.391,359.471 181.653,359.424 182.915,359.374 184.176,359.32 185.438,359.262 
 186.699,359.201 187.961,359.135 189.222,359.065 190.484,358.99 191.746,358.909 193.007,358.824 194.269,358.732 195.53,358.635 196.792,358.531 198.054,358.42 
 199.315,358.303 200.577,358.177 201.838,358.044 203.1,357.902 204.362,357.752 205.623,357.592 206.885,357.423 208.146,357.243 209.408,357.052 210.669,356.851 
 211.931,356.637 213.193,356.41 214.454,356.171 215.716,355.918 216.977,355.65 218.239,355.367 219.501,355.069 220.762,354.754 222.024,354.422 223.285,354.072 
 224.547,353.704 225.809,353.316 227.07,352.907 228.332,352.478 229.593,352.026 230.855,351.552 232.116,351.054 233.378,350.531 234.64,349.983 235.901,349.408 
 237.163,348.806 238.424,348.175 239.686,347.515 240.948,346.824 242.209,346.103 243.471,345.348 244.732,344.561 245.994,343.739 247.256,342.881 248.517,341.987 
 249.779,341.055 251.04,340.085 252.302,339.075 253.563,338.024 254.825,336.932 256.087,335.797 257.348,334.619 258.61,333.395 259.871,332.127 261.133,330.811 
 262.395,329.448 263.656,328.037 264.918,326.577 266.179,325.066 267.441,323.504 268.703,321.891 269.964,320.226 271.226,318.507 272.487,316.735 273.749,314.909 
 275.011,313.027 276.272,311.091 277.534,309.099 278.795,307.05 280.057,304.946 281.318,302.785 282.58,300.567 283.842,298.293 285.103,295.961 286.365,293.574 
 287.626,291.13 288.888,288.629 290.15,286.073 291.411,283.462 292.673,280.796 293.934,278.076 295.196,275.302 296.458,272.476 297.719,269.598 298.981,266.67 
 300.242,263.693 301.504,260.668 302.765,257.596 304.027,254.479 305.289,251.318 306.55,248.116 307.812,244.873 309.073,241.593 310.335,238.277 311.597,234.927 
 312.858,231.545 314.12,228.134 315.381,224.697 316.643,221.236 317.905,217.753 319.166,214.252 320.428,210.735 321.689,207.205 322.951,203.666 324.212,200.12 
 325.474,196.571 326.736,193.022 327.997,189.477 329.259,185.938

In [16]:
println("\n","end of program")


end of program
